In [38]:
from LambertSolarSystem import LambertSolarSystem
from PyKEP import AU, DAY2SEC, DAY2YEAR, MU_SUN, epoch
import PyKEP as pkp
import numpy as np

first_boost = 9594
#first_pos   = np.load('data/pos9594')
#first_times = np.load('data/times9594')
#first_vel   = np.load('data/vel9594')

vel_at_closest = 0
SEC2YEAR = 1./DAY2YEAR*DAY2SEC

system = LambertSolarSystem(87464)
t = 14.32
A = 4; B = 5
planets = np.array((A,B))
a, e, i, W, w, M  = system.getKeplerianElements()[:,planets]
M = system.getM(t)[planets]
kep_elements = np.array((a*AU,e,i,W,w,M))

mu_star = system.starMass * MU_SUN
mu_planet = system.mass[planets]  * MU_SUN
radius = system.radius[planets] * 1000
safe_radius = radius *1.2
planetA = pkp.planet.keplerian(epoch(0), kep_elements[:,0], mu_star, mu_planet[0], radius[0], safe_radius[0], 'four')
planetB = pkp.planet.keplerian(epoch(0), kep_elements[:,1], mu_star, mu_planet[1], radius[1], safe_radius[1], 'five')

time_array = np.linspace(5,7,100)
v_list = []
for time_use in time_array:
    t1 = epoch(0)            #relative to t0
    t2 = epoch(time_use/DAY2YEAR)   #relative to t0
    dt = (t2.mjd2000 - t1.mjd2000)*DAY2SEC


    r1, v1 = planetA.eph(t1)
    r2, v2 = planetB.eph(t2)
    
    l = pkp.lambert_problem(r1,r2,dt,mu_star)

    v1 = l.get_v1()
    vel =  np.linalg.norm(v1)/1000
    v_list.append((vel,time_use))
v_list = np.array(v_list)
print np.amin(v_list, axis = 0)
i = np.argmin(v_list, axis = 0)[0]
optimal_time = v_list[i,1]
print 'optimal time use:', v_list[i,1]



[ 17.325335   5.      ]
optimal time use: 6.23232323232


In [71]:
t1 = epoch(0)            #relative to t0
t2 = epoch(optimal_time/DAY2YEAR)   #relative to t0
dt = (t2.mjd2000 - t1.mjd2000)*DAY2SEC


r1, v1 = planetA.eph(t1)
r2, v2 = planetB.eph(t2)

l = pkp.lambert_problem(r1,r2,dt,mu_star)

v1 = np.array(l.get_v1())
posFunc = system.getPositionsFunction()
velFunc = system.getVelocityFunction()

planetPosA = posFunc(t)[:,4]
planetVelA = velFunc(t)[:,4]
planetPosB = posFunc(t+optimal_time)[:,5]
planetVelB = velFunc(t+optimal_time)[:,5]
print np.array(r1)
print planetPosA * AU
print v1[0,0:2], planetVelA

[  5.02073721e+11  -4.96073322e+11   0.00000000e+00]
[  4.99572182e+11  -4.98586299e+11]
[  7117.80101288  15795.70009767]
